# Chapter 38: Reference Counting

This notebook covers Python's primary memory management mechanism: reference counting. Every object in CPython maintains a count of how many references point to it. When that count drops to zero, the object is immediately deallocated.

## Key Concepts
- **`sys.getrefcount()`**: Inspect the current reference count of an object
- **Multiple references**: Assigning to new names increases the count
- **`del` statement**: Removes a reference, decreasing the count
- **Container references**: Lists, dicts, and other containers hold references to their elements
- **Immediate deallocation**: Objects are freed as soon as their reference count hits zero

## Section 1: How Reference Counting Works

In CPython, every object has an internal reference count. Each time a new name, container slot, or function argument refers to an object, the count goes up. When a reference is removed, the count goes down. At zero, the memory is reclaimed immediately.

In [ ]:
import sys

# Create a fresh object and check its reference count
a: list[int] = []
count: int = sys.getrefcount(a)

# getrefcount itself creates a temporary reference (its argument),
# so the reported count is always at least 2: the variable + the call
print(f"Reference count of a: {count}")
print(f"Count is >= 2: {count >= 2}")

In [ ]:
import sys

# Small integers are cached by CPython, so they have high ref counts
small_int: int = 42
print(f"Ref count of 42: {sys.getrefcount(small_int)}")

# A freshly created large integer has a low ref count
big_int: int = 123_456_789_000
print(f"Ref count of 123_456_789_000: {sys.getrefcount(big_int)}")

# None is referenced everywhere in the interpreter
print(f"Ref count of None: {sys.getrefcount(None)}")

## Section 2: Multiple References Increase the Count

When you assign an existing object to a new variable name, no copy is made. Instead, both names point to the same object and the reference count increases by one.

In [ ]:
import sys

a: list[int] = []
base: int = sys.getrefcount(a)
print(f"Base ref count (a only): {base}")

# Assign a second name pointing to the same object
b = a
after_b: int = sys.getrefcount(a)
print(f"After b = a:             {after_b}")
print(f"Increased by 1:          {after_b == base + 1}")

# Assign a third name
c = a
after_c: int = sys.getrefcount(a)
print(f"After c = a:             {after_c}")
print(f"Increased by 2 total:    {after_c == base + 2}")

# All three names refer to the exact same object
print(f"\na is b: {a is b}")
print(f"a is c: {a is c}")

## Section 3: `del` Decreases the Reference Count

The `del` statement removes a name binding. It does **not** directly delete the object -- it simply decreases the reference count by one. If other references remain, the object stays alive.

In [ ]:
import sys

a: list[int] = []
b = a
base: int = sys.getrefcount(a)
print(f"Ref count with a and b: {base}")

# Remove the reference held by b
del b
after_del: int = sys.getrefcount(a)
print(f"After del b:            {after_del}")
print(f"Decreased by 1:         {after_del == base - 1}")

# a still references the object, so it is not deallocated
print(f"\na still works: {a}")

In [ ]:
import sys

# Demonstrate multiple del operations
a: list[int] = [1, 2, 3]
b = a
c = a

print(f"Initial ref count: {sys.getrefcount(a)}")

del c
print(f"After del c:       {sys.getrefcount(a)}")

del b
print(f"After del b:       {sys.getrefcount(a)}")

# At this point only 'a' holds a reference
# If we del a, the object would be immediately freed
print(f"\nObject still alive via a: {a}")

## Section 4: Container References

When an object is placed inside a container (list, dict, set, tuple, etc.), the container holds a reference to it. This increases the object's reference count.

In [ ]:
import sys

obj: list[int] = [10, 20, 30]
base: int = sys.getrefcount(obj)
print(f"Base ref count:           {base}")

# Placing obj in a list increases its ref count
container: list[list[int]] = [obj]
in_list: int = sys.getrefcount(obj)
print(f"After adding to list:     {in_list}")
print(f"Increased by 1:           {in_list == base + 1}")

# Adding to a second container increases it again
another: list[list[int]] = [obj]
in_two: int = sys.getrefcount(obj)
print(f"After adding to 2nd list: {in_two}")
print(f"Increased by 2 total:     {in_two == base + 2}")

In [ ]:
import sys

# Dictionaries also hold references to values (and keys)
obj: list[str] = ["hello"]
base: int = sys.getrefcount(obj)

data: dict[str, list[str]] = {"key": obj}
in_dict: int = sys.getrefcount(obj)
print(f"Base ref count:        {base}")
print(f"After adding to dict:  {in_dict}")
print(f"Increased by 1:        {in_dict == base + 1}")

# Removing from the container decreases it
del data["key"]
after_remove: int = sys.getrefcount(obj)
print(f"After removing key:    {after_remove}")
print(f"Back to base:          {after_remove == base}")

## Section 5: Function Arguments and Reference Counts

When you pass an object as a function argument, the parameter name creates an additional reference for the duration of the call. This is why `sys.getrefcount()` always reports at least 2.

In [ ]:
import sys


def show_refcount(obj: object) -> None:
    """Print the reference count inside a function call."""
    # obj is an additional reference (the parameter)
    # sys.getrefcount adds one more (its own argument)
    count: int = sys.getrefcount(obj)
    print(f"  Inside function: {count}")


a: list[int] = [1, 2, 3]
print(f"Before function call: {sys.getrefcount(a)}")
show_refcount(a)
print(f"After function call:  {sys.getrefcount(a)}")

# The count goes up inside the function (parameter + getrefcount arg)
# and comes back down after the function returns

## Section 6: Practical Patterns

Understanding reference counting helps you reason about object lifetimes and avoid keeping objects alive unintentionally.

In [ ]:
import sys


def track_references(label: str, obj: object) -> int:
    """Print and return the current reference count for an object."""
    # Subtract 1 for the getrefcount argument itself
    # Subtract 1 for the function parameter
    estimated: int = sys.getrefcount(obj) - 2
    print(f"{label}: ~{estimated} external references")
    return estimated


data: list[int] = [1, 2, 3]

track_references("After creation", data)

alias = data
track_references("After alias", data)

cache: dict[str, list[int]] = {"numbers": data}
track_references("After cache", data)

del alias
track_references("After del alias", data)

del cache
track_references("After del cache", data)

In [ ]:
import sys

# Demonstrate that reassigning a variable releases the old reference
a: list[int] = [100, 200, 300]
obj_id: int = id(a)
print(f"Original id: {obj_id}")
print(f"Ref count:   {sys.getrefcount(a)}")

# Reassigning 'a' releases the reference to the old list
# If no other references exist, the old list is freed immediately
a = [400, 500, 600]
new_id: int = id(a)
print(f"\nNew id:      {new_id}")
print(f"Same object? {obj_id == new_id}")
print(f"Ref count:   {sys.getrefcount(a)}")

## Section 7: Limitations of Reference Counting

Reference counting handles most deallocation, but it cannot detect **circular references** -- objects that reference each other. This is where Python's garbage collector (covered in the next notebook) steps in.

In [ ]:
import sys

# Create two objects that reference each other
a: dict[str, object] = {}
b: dict[str, object] = {}
a["partner"] = b
b["partner"] = a

print(f"Ref count of a: {sys.getrefcount(a)}")
print(f"Ref count of b: {sys.getrefcount(b)}")

# Even after deleting both names, the objects keep each other alive
# through their mutual references. Reference counting alone
# cannot free them -- the garbage collector must intervene.
print("\nCircular references prevent ref count from reaching zero.")
print("The gc module handles this case (see next notebook).")

## Summary

### Reference Counting Basics
- Every CPython object has an internal reference count
- `sys.getrefcount(obj)` returns the count (includes +1 for its own argument)
- Objects are deallocated immediately when their reference count reaches zero

### What Increases the Count
- Assigning to a new variable: `b = a`
- Placing in a container: `my_list.append(obj)`
- Passing as a function argument

### What Decreases the Count
- `del name` removes a name binding
- Reassigning a variable to a different object
- Removing from a container
- A local variable going out of scope when a function returns

### Limitations
- Cannot detect circular references (objects referencing each other)
- The `gc` module handles cycle detection (covered in the next notebook)
- `sys.getrefcount()` is CPython-specific and may not work on other implementations